In [1]:
## ABRIMOS EL DATASET

import pickle
import time
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Cargar el dataset 
with open("dataset_dhl.pkl", "rb") as f:
    data = pickle.load(f)


In [2]:

## AÑADIMOS LAS COORDENADAS ##

# Metemos las coordenadas que ya conocemos (las que no podemos calcular con geopy)
coordenadas_fijas = {
    "Atlante - Atlante - Nieves Sant Esteve": (41.504070650751906, 1.883253906745552),
    "Atlante - Nieves Zona Franca":( 41.33194049366559, 2.134952349708804),
    "IONITY Montcada i Reixac": (41.491422605263075, 2.1823508377517684),
    "IONITY Barcera del Valles": (41.51182358577195, 2.1345502627626303),
    "Ctra. Molins de Rei, 251, Km. 8, 08191 Rubí, Barcelona": (41.46930679260032, 2.029808746545829),
    "Carrer Sakura, 16B, NAVE 16A, 08272 Sant Fruitós de Bages, Barcelona": (41.77803590470732, 1.867141253552206),
    "Avinguda Can Montcau, 1-3-5, 08186 Lliçà d'Amunt, Barcelona": (41.60289038509304, 2.2569771240231096),
    "DHL FREIGHT SPAIN S.L, C/ Atlantic 109-111  (ZAL)": (41.548266507563135, 2.1687455645705684)
}

# Inicializar geolocalizador 
geolocator = Nominatim(user_agent="geoapi")

# Añadir coordenadas 
for d in data:
    direccion = d.get("direccion", "").strip()
    nombre = d.get("nombre", "").strip()

    # Caso 1: coordenadas conocidas
    if direccion in coordenadas_fijas:
        d["coordenadas"] = coordenadas_fijas[direccion]

    elif nombre in coordenadas_fijas:
        d["coordenadas"] = coordenadas_fijas[nombre]

    # Caso 2: intentar buscar con geopy
    elif direccion:
        try:
            location = geolocator.geocode(direccion)
            if location:
                d["coordenadas"] = (location.latitude, location.longitude)
            else:
                d["coordenadas"] = None
                print(f"No se encontró la dirección: {nombre}")
        except Exception as e:
            print(f"Error al buscar {nombre}: {e}")
            d["coordenadas"] = None
        time.sleep(1)  # Evita bloqueo por exceso de peticiones

    else:
        d["coordenadas"] = None
        print(f" No hay dirección para: {nombre}")





In [3]:
## MATRIZ DE DISTANCIAS 
n = len(data)
distancias = [[0]*n for _ in range(n)]

for i in range(n):
    for j in range(n):
        if i != j and data[i]["coordenadas"] and data[j]["coordenadas"]:
            dist = geodesic(data[i]["coordenadas"], data[j]["coordenadas"]).km
            distancias[i][j] = round(dist, 2)
        else:
            distancias[i][j] = None if i != j else 0


#Mostrar matriz
for fila in distancias:
    print(fila)

    

[0, 16.64, 36.87, 24.07, 23.79, 14.54, 5.92, 9.94, 12.98, 17.61, 12.83]
[16.64, 0, 52.74, 36.68, 37.16, 27.87, 11.46, 23.03, 22.88, 8.76, 23.96]
[36.87, 52.74, 0, 37.84, 35.68, 35.81, 41.3, 37.02, 41.27, 54.35, 30.46]
[24.07, 36.68, 37.84, 0, 2.2, 9.54, 29.41, 14.37, 13.86, 31.77, 33.05]
[23.79, 37.16, 35.68, 2.2, 0, 9.47, 29.32, 14.42, 14.6, 32.71, 32.06]
[14.54, 27.87, 35.81, 9.54, 9.47, 0, 19.9, 4.95, 6.41, 24.19, 24.33]
[5.92, 11.46, 41.3, 29.41, 29.32, 19.9, 0, 15.04, 17.02, 14.83, 13.6]
[9.94, 23.03, 37.02, 14.37, 14.42, 4.95, 15.04, 0, 4.59, 19.98, 21.0]
[12.98, 22.88, 41.27, 13.86, 14.6, 6.41, 17.02, 4.59, 0, 18.15, 25.01]
[17.61, 8.76, 54.35, 31.77, 32.71, 24.19, 14.83, 19.98, 18.15, 0, 28.43]
[12.83, 23.96, 30.46, 33.05, 32.06, 24.33, 13.6, 21.0, 25.01, 28.43, 0]


In [4]:
## MATRIZ DE TIEMPOS (d):
# Vamos a suponer que el camion va a una velocidad media de 70km/h
tiempos = [[0]*n for _ in range(n)]
velocidad=70
for i in range(len(distancias)):
    for j in range(len(distancias)):
        tiempos[i][j]=distancias[i][j]/70*60 # en minutos
for filas in tiempos:
    print(filas)  

[0.0, 14.262857142857143, 31.60285714285714, 20.63142857142857, 20.39142857142857, 12.462857142857143, 5.074285714285715, 8.52, 11.125714285714286, 15.094285714285714, 10.997142857142856]
[14.262857142857143, 0.0, 45.205714285714286, 31.44, 31.851428571428567, 23.888571428571428, 9.822857142857144, 19.740000000000002, 19.611428571428572, 7.508571428571428, 20.537142857142857]
[31.60285714285714, 45.205714285714286, 0.0, 32.434285714285714, 30.58285714285714, 30.694285714285712, 35.4, 31.731428571428573, 35.37428571428572, 46.58571428571429, 26.10857142857143]
[20.63142857142857, 31.44, 32.434285714285714, 0.0, 1.885714285714286, 8.177142857142856, 25.208571428571428, 12.317142857142857, 11.879999999999999, 27.23142857142857, 28.328571428571426]
[20.39142857142857, 31.851428571428567, 30.58285714285714, 1.885714285714286, 0.0, 8.117142857142857, 25.13142857142857, 12.36, 12.514285714285714, 28.037142857142857, 27.48]
[12.462857142857143, 23.888571428571428, 30.694285714285712, 8.1771428

In [5]:
## VECTOR DE PRECIO CARGA (w)
import re
precio_ionity = 0.61
precio_atlante = 0.64
kwh = 300


for d in data:
    if re.search("IONITY",d["nombre"]):
        d["w"]=kwh*precio_ionity
    elif re.search("Atlante",d["nombre"]):
         d["w"]=kwh*precio_atlante
    else:
         d["w"]=0
       


In [ ]:
# MATRIZ DE COSTES (cij)
# cij=dist*precio
precio=0.4
c=[[0]*len(data) for _ in range(len(data))]
for i in range(len(data)):
    for j in range(len(data)):
        c[i][j]=distancias[i][j]*0.4
for filas in c:
    print(filas)
   


[0.0, 6.656000000000001, 14.748, 9.628, 9.516, 5.816, 2.368, 3.976, 5.192, 7.0440000000000005, 5.132000000000001]
[6.656000000000001, 0.0, 21.096000000000004, 14.672, 14.863999999999999, 11.148000000000001, 4.5840000000000005, 9.212000000000002, 9.152, 3.504, 9.584000000000001]
[14.748, 21.096000000000004, 0.0, 15.136000000000003, 14.272, 14.324000000000002, 16.52, 14.808000000000002, 16.508000000000003, 21.740000000000002, 12.184000000000001]
[9.628, 14.672, 15.136000000000003, 0.0, 0.8800000000000001, 3.816, 11.764000000000001, 5.748, 5.5440000000000005, 12.708, 13.219999999999999]
[9.516, 14.863999999999999, 14.272, 0.8800000000000001, 0.0, 3.7880000000000003, 11.728000000000002, 5.768000000000001, 5.84, 13.084000000000001, 12.824000000000002]
[5.816, 11.148000000000001, 14.324000000000002, 3.816, 3.7880000000000003, 0.0, 7.96, 1.9800000000000002, 2.564, 9.676000000000002, 9.732]
[2.368, 4.5840000000000005, 16.52, 11.764000000000001, 11.728000000000002, 7.96, 0.0, 6.016, 6.808, 5.93

NameError: name 'w' is not defined

In [7]:
## CREAR NUEVO DATA SET
dataset_actualizado = {
    "data": data,
    "distancias": distancias, 
    "tiempos": tiempos,
    "costes":c
}

with open("dataset_actualizado.pkl", "wb") as f:
    pickle.dump(dataset_actualizado, f)

